In [1]:
import pandas as pd

df=pd.read_excel("data.xls")
df.head()

,ID,Gender,da025,da049,died,fall2018,Conclusion,ec001,address,education2015,...,treatment_psychiatric,treatment_pain,treatment_dyslioidemia,treatment_lungdisease,treatment_liverdisease,treatment_heartdisease,treatment_kidneydisease,treatment_digestivedisease,treatment_memorydisease,treatment_arthritisdisease
0,101791255002,2,2.0,7.5,0.0,2.0,3,1.0,1.0,NaN,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
1,101791257001,1,2.0,6.0,0.0,2.0,3,1.0,1.0,NaN,...,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0
2,101791258001,2,2.0,7.5,0.0,2.0,3,3.0,1.0,4.0,...,0.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,101791258002,1,2.0,5.0,0.0,2.0,3,3.0,1.0,5.0,...,0.0,2.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
4,101791259001,1,2.0,4.0,0.0,2.0,3,1.0,1.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
df.isnull().sum().sum()

27806

In [12]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, RidgeClassifier
from sklearn.model_selection import train_test_split

class MiceImputer(object):

    def __init__(self, seed_values = True, seed_strategy="mean", copy=True):
        self.strategy = seed_strategy # seed_strategy in ['mean','median','most_frequent', 'constant']
        self.seed_values = seed_values # seed_values = False initializes missing_values using not_null columns
        self.copy = copy
        self.imp = SimpleImputer(strategy=self.strategy, copy=self.copy)

    def fit_transform(self, X, method = 'Linear', iter = 5, verbose = True):
        
        # Why use Pandas?
        # http://gouthamanbalaraman.com/blog/numpy-vs-pandas-comparison.html
        # Pandas < Numpy if X.shape[0] < 50K
        # Pandas > Numpy if X.shape[0] > 500K
        
        # Data necessary for masking missing-values after imputation
        null_cols = X.columns[X.isna().any()].tolist()
        null_X = X.isna()[null_cols]
      
        ### Initialize missing_values
        
        if self.seed_values:
            
            # Impute all missing values using SimpleImputer 
            if verbose:
                print('Initilization of missing-values using SimpleImputer')
            new_X = pd.DataFrame(self.imp.fit_transform(X))
            new_X.columns = X.columns
            new_X.index = X.index
            
        else:
   
            # Initialize a copy based on value of self.copy
            if self.copy:
                new_X = X.copy()
            else:
                new_X = X
                
            not_null_cols = X.columns[X.notna().any()].tolist()
            
            if verbose:
                print('Initilization of missing-values using regression on non-null columns')
               
            for column in null_cols:
                
                null_rows = null_X[column]
                train_x = new_X.loc[~null_rows, not_null_cols]
                test_x = new_X.loc[null_rows, not_null_cols]
                train_y = new_X.loc[~null_rows, column]
                
                if X[column].nunique() > 2:
                    m = LinearRegression(n_jobs = -1)
                    m.fit(train_x, train_y)
                    new_X.loc[null_rows,column] = pd.Series(m.predict(test_x))
                    not_null_cols.append(column)
                    
                elif X[column].nunique() == 2:
                    m = LogisticRegression(n_jobs = -1, solver = 'lbfgs')
                    m.fit(train_x, train_y)
                    new_X.loc[null_rows,column] = pd.Series(m.predict(test_x))
                    not_null_cols.append(column)
        
        ### Begin iterations of MICE
        
        model_score = {}
        
        for i in range(iter):
            if verbose:
                print('Beginning iteration ' + str(i) + ':')
                
            model_score[i] = []
            
            for column in null_cols:
                
                null_rows = null_X[column]                
                not_null_y = new_X.loc[~null_rows, column]
                not_null_X = new_X[~null_rows].drop(column, axis = 1)
                
                train_x, val_x, train_y, val_y = train_test_split(not_null_X, not_null_y, test_size=0.33, random_state=42)  
                test_x = new_X.drop(column, axis = 1)
                  
                if new_X[column].nunique() > 2:
                    if method == 'Linear':
                        m = LinearRegression(n_jobs = -1)
                    elif method == 'Ridge':
                        m = Ridge()
                        
                    m.fit(train_x, train_y)
                    model_score[i].append(m.score(val_x, val_y))
                    new_X.loc[null_rows,column] = pd.Series(m.predict(test_x))
                    if verbose:
                        print('Model score for ' + str(column) + ': ' + str(m.score(val_x, val_y)))
                    
                elif new_X[column].nunique() == 2:
                    if method == 'Linear':
                        m = LogisticRegression(n_jobs = -1, solver = 'lbfgs')
                    elif method == 'Ridge':
                        m = RidgeClassifier()
                        
                    m.fit(train_x, train_y)
                    model_score[i].append(m.score(val_x, val_y))
                    new_X.loc[null_rows,column] = pd.Series(m.predict(test_x))
                    if verbose:
                        print('Model score for ' + str(column) + ': ' + str(m.score(val_x, val_y)))
                
            if model_score[i] == []:
                model_score[i] = 0
            else:
                model_score[i] = sum(model_score[i])/len(model_score[i])

        return new_X

In [13]:
imp=MiceImputer()
df_new=imp.fit_transform(df)

Initilization of missing-values using SimpleImputer
Beginning iteration 0:
Model score for da025: 0.0057041679782937615
Model score for da049: 0.05275358794494789
Model score for died: 0.9999999819839274
Model score for fall2018: 0.9999999999945516
Model score for ec001: 0.05754949091610051
Model score for address: 0.1560891482218284
Model score for education2015: 0.3097940410536266
Model score for Da005_1_: 0.12161928577678527
Model score for Da005_2_: 0.0999400490501885
Model score for Da005_3_: 0.10809487596434764
Model score for Da005_4_: 0.23797803783024796
Model score for Da005_5_: -0.0018720237660334593
Model score for Da007_1_: 0.8320198668397532
Model score for Da007_2_: 0.7948537244139815
Model score for Da007_3_: 0.8241765395013594
Model score for Da007_4_: 0.5797960216568291
Model score for Da007_5_: 0.7576771569710844
Model score for Da007_6_: 0.628826737668621
Model score for Da007_7_: 0.8039469095791933
Model score for Da007_8_: 0.7082495763103287
Model score for Da007_9

Model score for died: 0.2847414843846402
Model score for fall2018: 0.9999999999984148
Model score for ec001: 0.05935236378369024
Model score for address: 0.1712107520992504
Model score for education2015: 0.31150187405003604
Model score for Da005_1_: 0.12816122011256004
Model score for Da005_2_: 0.10568045121686065
Model score for Da005_3_: 0.11071566275658085
Model score for Da005_4_: 0.24016475627856282
Model score for Da005_5_: 0.00787949838463553
Model score for Da007_1_: 0.8809236198584697
Model score for Da007_2_: 0.9046466548309564
Model score for Da007_3_: 0.8941374484001633
Model score for Da007_4_: 0.5792957614791805
Model score for Da007_5_: 0.9140404175823247
Model score for Da007_6_: 0.9391999841662292
Model score for Da007_7_: 0.8930572546825173
Model score for Da007_8_: 0.9116843337518428
Model score for Da007_9_: 0.9315824767290325
Model score for Da007_10_: 0.8918467186652367
Model score for Da007_11_: 0.9535901546494745
Model score for Da007_12_: 0.9317413846052428
Mod

In [14]:
df_new.isnull().sum().sum()

0

In [15]:
df_new.to_csv("inputed_data.csv")